In [0]:
df= spark.readStream.table('professional.superstore_bronze.gmart_bronze')

In [0]:
df.printSchema()

In [0]:
from pyspark.sql.functions import *

In [0]:

from pyspark.sql.functions import to_date, to_timestamp, expr, col

df = (
    df.withColumn("Quantity", expr("try_cast(Quantity AS int)"))
      .withColumn("Sales", expr("try_cast(Sales AS double)"))
      .withColumn("Discount", expr("try_cast(Discount AS double)"))
      .withColumn("Profit", expr("try_cast(Profit AS double)"))
      .withColumn("Postal_Code", expr("try_cast(Postal_Code AS int)"))
      
      # Parse from M/d/yyyy → DATE
      .withColumn("Ship_Date", to_date(col("Ship_Date"), "M/d/yyyy"))
      .withColumn("Order_Date", to_date(col("Order_Date"), "M/d/yyyy"))
      
      # Convert DATE → TIMESTAMP
      .withColumn("Ship_Date", to_timestamp(col("Ship_Date")))
      .withColumn("Order_Date", to_timestamp(col("Order_Date")))

      .where(col("Quantity") >= 1)
      .na.drop(subset=["City", "Postal_Code"])
      
)





In [0]:
from pyspark.sql.functions import count


df.withWatermark("ingestion_time", "1 hours")\
    .groupBy(
          "Customer_ID",
          "Customer_Name",
          "Country",
          "City",
          "State",
          "Postal_Code",
          "Region"
      )\
      .agg(count("*"))\
      .select(
          "Customer_ID",
          "Customer_Name",
          "Country",
          "City",
          "State",
          "Postal_Code",
          "Region"
      )\
      .writeStream\
      .trigger(availableNow=True)\
      .format("delta")\
      .outputMode("complete") \
      .option("checkpointLocation", "/Volumes/professional/default/superstore/schema_sil")\
      .toTable("professional.superstore_silver.gmart_customers")


In [0]:
display(df.groupBy("Order_ID","Order_Date","Ship_Date","Ship_Mode","Product_ID").agg(count("*")))